# RCU Metrics Tests

In [1]:
%load_ext autoreload
%autoreload 2
import os
os.chdir("/scratch/ewalt/pdm/rs-uncertainty")
# from src.rcu_metrics import StratifiedRCUSubset, StratifiedRCU, nan_frac, weighted_avg
from src.metrics import StratifiedRCUSubset, StratifiedRCU, nan_frac, weighted_avg
import rasterio
import argparse
import yaml
import numpy as np
from tqdm import tqdm
from pathlib import Path
import pandas as pd

In [2]:
nan_frac(np.array([np.nan, np.nan, np.nan]))

1.0

In [3]:
def pjoin(*subs): return Path(os.path.abspath(os.path.join(*subs)))
def get_variance_bounds(cfg, N=2, normalize_variance=False):
    projects = []
    if normalize_variance:
        # Load standardization data
        with pjoin(cfg["pkl_dir"], "stats.yaml").open("r", encoding="utf-8") as f:
            stats = yaml.safe_load(f)
        labels_mean = np.array(stats["labels_stats"]["mean"])
        labels_std = np.array(stats["labels_stats"]["std"])
    for k, v in cfg.items():
        if k.endswith("_dir"): cfg[k] = Path(v)
    projects = cfg["projects_east"]+cfg["projects_west"]+cfg["projects_north"]
    # loop on projects to get variance bounds
    print(f"Computing variance bounds in {cfg['pkl_dir']}...")
    lo_variance = np.full((5,), np.inf)
    hi_variance = np.full((5,), -np.inf)
    variance_files = []
    # for variance_file in tqdm(list(cfg["prediction_dir"].glob("*_variance.tif"))):
    for variance_file in tqdm(list(cfg["prediction_dir"].glob("*_variance.tif"))[:N]): # Debug
        if variance_file.stem.split("_")[0] not in projects: continue
        variance_files.append(variance_file)
        with rasterio.open(variance_file) as fh:
            variance = fh.read(fh.indexes)
        variance_flat = variance.reshape(5, -1)
        hi = np.nanmax(variance_flat, axis=1)
        lo = np.nanmin(variance_flat, axis=1)
        hi_variance[hi>hi_variance] = hi[hi>hi_variance]
        lo_variance[lo<lo_variance] = lo[lo<lo_variance]
    print("Variances lower bound:", lo_variance.tolist())
    print("Variances upper bound:", hi_variance.tolist())
    # initialize RCU metrics
    if normalize_variance:
        return lo_variance/labels_std**2, hi_variance/labels_std**2, variance_files
    else:
        return lo_variance, hi_variance, variance_files
def get_projects(cfg, variance_files, normalize_variance=False):
    projects = []
    # Load standardization data
    with pjoin(cfg["pkl_dir"], "stats.yaml").open("r", encoding="utf-8") as f:
        stats = yaml.safe_load(f)
    labels_mean = np.array(stats["labels_stats"]["mean"]).reshape(5,1,1)
    labels_std = np.array(stats["labels_stats"]["std"]).reshape(5,1,1)
    # compute stats online
    for variance_file in tqdm(variance_files):
        # load data
        project = variance_file.stem.split('_')[0]
        # if project not in projects: continue
        with rasterio.open(pjoin(cfg['prediction_dir'], f"{project}_mean.tif")) as fh:
            mean = fh.read(fh.indexes)
        with rasterio.open(variance_file) as fh:
            variance = fh.read(fh.indexes)
        with rasterio.open(pjoin(cfg['gt_dir'], f"{project}.tif")) as fh:
            gt = fh.read(fh.indexes)
            gt[2] /= 100 # Cover/Dens normalization!!
            gt[4] /= 100
        # standardize
        mean[[0,1]] = (mean[[0,1]]-labels_mean[[0,1]])/labels_std[[0,1]]
        gt[[0,1]] = (gt[[0,1]]-labels_mean[[0,1]])/labels_std[[0,1]]
        if normalize_variance: variance /= labels_std**2
        projects.append((project, gt, mean, variance))
    return projects    
def res2df(res, cfg):
    R = {}
    for eid, eres in res.items():
        for metric_name, metric_info in eres.items():
            R[(eid, metric_name)] = metric_info["values"]
            R[(eid, f"ause-{metric_name}")] = metric_info["ause"]
    return pd.DataFrame(R, index=cfg["variable_names"]).T
def test(metrics, lo_variance, hi_variance, projects, cfg, N=2):
    print("creating")
    rcu = rcu = StratifiedRCUSubset(
        metric_names=metrics,
        num_variables=len(cfg["data_bands"]),
        # num_groups=len(projects),
        num_groups=N, # Debug
        num_bins=cfg["num_bins"],
        lo_variance=lo_variance,
        hi_variance=hi_variance
    )
    print("adding")
    results = {}
    for i, project in enumerate(projects):
        print("adding", project[0])
        rcu.add_project(*project)
        results[project[0]] = rcu.get_subset([project[0]])
    results["global"] = rcu.get()
    return res2df(results, cfg), rcu

In [4]:
with Path("./config/evaluate_testset/baseline.yaml").open("r", encoding="utf-8") as f:
    cfg = yaml.safe_load(f)
cfg["num_bins"] = 50
lo_variance, hi_variance, variance_files = get_variance_bounds(cfg)
projects = get_projects(cfg, variance_files)

  0%|                                                | 0/2 [00:00<?, ?it/s]

Computing variance bounds in data/2023-04-05_18-58-33...


100%|████████████████████████████████████████| 2/2 [00:01<00:00,  1.45it/s]


Variances lower bound: [0.4428499639034271, 0.08763699978590012, 0.0004826262593269348, 0.0001668027980485931, 4.102319962839829e-06]
Variances upper bound: [268.0447998046875, 106.31964111328125, 0.11541682481765747, 0.0358952134847641, 0.18080325424671173]


100%|████████████████████████████████████████| 2/2 [00:06<00:00,  2.39s/it]


### Error Metrics [pOK]

In [5]:
res, rcu = test(["mse", "rmse", "mae", "mbe"], lo_variance, hi_variance, projects, cfg)

creating
adding
adding 439
[rcu] adding StratifiedMSE for 439
[rcu] adding StratifiedRMSE for 439
[rcu] adding StratifiedMAE for 439
[rcu] adding StratifiedMBE for 439
[rcu] getting StratifiedMSE for ['439']
[rcu] getting StratifiedRMSE for ['439']
[rcu] getting StratifiedMAE for ['439']
[rcu] getting StratifiedMBE for ['439']
adding 471
[rcu] adding StratifiedMSE for 471
[rcu] adding StratifiedRMSE for 471
[rcu] adding StratifiedMAE for 471
[rcu] adding StratifiedMBE for 471
[rcu] getting StratifiedMSE for ['471']
[rcu] getting StratifiedRMSE for ['471']
[rcu] getting StratifiedMAE for ['471']
[rcu] getting StratifiedMBE for ['471']
[rcu] getting StratifiedMSE globally
[rcu] getting StratifiedRMSE globally
[rcu] getting StratifiedMAE globally
[rcu] getting StratifiedMBE globally


In [6]:
res

P95     MeanH      Dens      Gini     Cover
439    mse        0.325619  0.345869  0.014802  0.002589  0.019557
       ause-mse   0.307903  0.324511  0.013128  0.002400  0.017197
       rmse       0.549281  0.568683  0.116070  0.048601  0.128315
       ause-rmse  0.523933  0.540405  0.107569  0.045919  0.118036
       mae        0.435301  0.450296  0.092023  0.037767  0.101679
       ause-mae   0.414987  0.427609  0.085085  0.035688  0.093075
       mbe       -0.042112 -0.027191 -0.004988 -0.003574 -0.015277
       ause-mbe  -0.040283 -0.025789 -0.003595 -0.003423 -0.013370
471    mse        0.267499  0.256655  0.014136  0.002227  0.022606
       ause-mse   0.254306  0.242438  0.013027  0.002069  0.020753
       rmse       0.497782  0.481926  0.113616  0.044666  0.143245
       ause-rmse  0.476129  0.459646  0.106805  0.042359  0.133938
       mae        0.387137  0.376022  0.089598  0.034760  0.113103
       ause-mae   0.370243  0.358502  0.084124  0.032967  0.105471
       mbe        0.033394  0.029016  0.000894  0.000052  0.006670
       ause-mbe   0.031830  0.027660  0.000943  0.000015  0.005962
global mse        0.275376  0.268746  0.014226  0.002276  0.022193
       ause-mse   0.272725  0.267529  0.013505  0.002218  0.020252
       rmse       0.504762  0.493684  0.113949  0.045200  0.141221
       ause-rmse  0.491503  0.482542  0.108774  0.043827  0.131430
       mae        0.393665  0.386088  0.089927  0.035168  0.111555
       ause-mae   0.383314  0.377721  0.085812  0.034086  0.103479
       mbe        0.023160  0.021398  0.000097 -0.000439  0.003696
       ause-mbe   0.024351  0.022121  0.000220 -0.000377  0.003483

### NLL [pOK]

In [7]:
res, rcu = test(["nll"], lo_variance, hi_variance, projects, cfg)

creating
adding
adding 439
[rcu] adding StratifiedNLL for 439
[rcu] getting StratifiedNLL for ['439']
adding 471
[rcu] adding StratifiedNLL for 471
[rcu] getting StratifiedNLL for ['471']
[rcu] getting StratifiedNLL globally


In [8]:
res

P95     MeanH      Dens      Gini     Cover
439    nll       0.962645  0.672108 -1.636143 -2.537406 -1.618837
       ause-nll  0.918872  0.635548 -1.602038 -2.450142 -1.597655
471    nll       0.742455  0.361909 -1.559775 -2.579001 -1.369919
       ause-nll  0.709931  0.341542 -1.520492 -2.485340 -1.348215
global nll       0.772297  0.403950 -1.570125 -2.573364 -1.403655
       ause-nll  0.752109  0.405491 -1.511714 -2.458143 -1.388013

### UCE, ENCE [pOK]

In [9]:
res, rcu = test(["uce", "ence"], lo_variance, hi_variance, [projects[1], projects[0]], cfg)

creating
adding
adding 471
[rcu] adding StratifiedUCE for 471
[rcu] adding StratifiedENCE for 471
[rcu] getting StratifiedUCE for ['471']
[rcu] getting StratifiedENCE for ['471']
adding 439


/scratch/ewalt/pdm/rs-uncertainty/src/metrics.py:250: RuntimeWarning: invalid value encountered in true_divide
  result = np.abs(result)/np.sqrt(np.nansum(histogram*arrs[0], axis=self.groups_axis, keepdims=True))


[rcu] adding StratifiedUCE for 439
[rcu] adding StratifiedENCE for 439
[rcu] getting StratifiedUCE for ['439']
[rcu] getting StratifiedENCE for ['439']
[rcu] getting StratifiedUCE globally
[rcu] getting StratifiedENCE globally


In [10]:
res

P95     MeanH      Dens      Gini     Cover
471    uce        4.826016  2.230569  0.007389  0.000942  0.011216
       ause-uce   4.585782  2.104669  0.006830  0.000876  0.010356
       ence       0.000017  0.000014  0.000007  0.000007  0.000004
       ause-ence  0.000012  0.000010  0.000004  0.000003  0.000003
439    uce        7.349940  3.789855  0.005263  0.000765  0.007109
       ause-uce   6.944419  3.544948  0.004689  0.000701  0.006343
       ence       0.000129  0.000124  0.000045  0.000062  0.000036
       ause-ence  0.000084  0.000076  0.000022  0.000027  0.000018
global uce        5.168084  2.441899  0.007100  0.000916  0.010655
       ause-uce   5.084292  2.371115  0.005204  0.000681  0.008439
       ence       0.000017  0.000016  0.000007  0.000006  0.000005
       ause-ence  0.000006  0.000005  0.000002  0.000001  0.000001

### CI90 [pOK]

In [11]:
res, rcu = test(["ci90_accs"], lo_variance, hi_variance, projects, cfg)

creating
adding
adding 439
[rcu] adding StratifiedCIAccuracy for 439
[rcu] getting StratifiedCIAccuracy for ['439']
adding 471
[rcu] adding StratifiedCIAccuracy for 471
[rcu] getting StratifiedCIAccuracy for ['471']
[rcu] getting StratifiedCIAccuracy globally


In [12]:
res

P95     MeanH      Dens      Gini     Cover
439    ci90_accs       0.999993  0.999857  0.821685  0.852386  0.853149
       ause-ci90_accs  0.959993  0.959859  0.788972  0.818340  0.821228
471    ci90_accs       0.999852  0.999617  0.763954  0.808580  0.788883
       ause-ci90_accs  0.959858  0.959630  0.734815  0.776314  0.760625
global ci90_accs       0.999871  0.999649  0.771778  0.814517  0.797593
       ause-ci90_accs  0.959874  0.959667  0.740188  0.780727  0.769884

### AUCE [pOK]

In [13]:
res, rcu = test(["auce"], lo_variance, hi_variance, projects, cfg)

creating
adding
adding 439
[rcu] adding StratifiedAUCE for 439
[rcu] getting StratifiedAUCE for ['439']
adding 471
[rcu] adding StratifiedAUCE for 471
[rcu] getting StratifiedAUCE for ['471']
[rcu] getting StratifiedAUCE globally


In [14]:
res

P95     MeanH      Dens      Gini     Cover
439    auce       0.368993  0.317995  0.067401  0.036826  0.065870
       ause-auce  0.354288  0.305226  0.064118  0.035330  0.061623
471    auce       0.355436  0.298897  0.104811  0.070676  0.097014
       ause-auce  0.341217  0.286885  0.099554  0.067790  0.091232
global auce       0.357274  0.301485  0.099741  0.066089  0.092793
       ause-auce  0.342780  0.289376  0.096202  0.064276  0.086486

In [15]:
res

P95     MeanH      Dens      Gini     Cover
439    auce       0.368993  0.317995  0.067401  0.036826  0.065870
       ause-auce  0.354288  0.305226  0.064118  0.035330  0.061623
471    auce       0.355436  0.298897  0.104811  0.070676  0.097014
       ause-auce  0.341217  0.286885  0.099554  0.067790  0.091232
global auce       0.357274  0.301485  0.099741  0.066089  0.092793
       ause-auce  0.342780  0.289376  0.096202  0.064276  0.086486

### C_v [pOK]

In [16]:
res, rcu = test(["cv"], lo_variance, hi_variance, projects, cfg)

creating
adding
adding 439
[rcu] adding StratifiedCv for 439
[rcu] getting StratifiedCv for ['439']
adding 471
[rcu] adding StratifiedCv for 471
[rcu] getting StratifiedCv for ['471']
[rcu] getting StratifiedCv globally


In [17]:
res

P95     MeanH      Dens      Gini     Cover
439    cv       0.091181  0.077670  0.042320  0.059102  0.074839
       ause-cv  0.088301  0.075879  0.043628  0.057862  0.080463
471    cv       0.149803  0.150098  0.059911  0.078631  0.067806
       ause-cv  0.144645  0.145961  0.060239  0.076739  0.071390
global cv       0.140786  0.138462  0.057254  0.075555  0.068805
       ause-cv  0.125688  0.123544  0.053705  0.068155  0.068181

In [18]:
res

P95     MeanH      Dens      Gini     Cover
439    cv       0.091181  0.077670  0.042320  0.059102  0.074839
       ause-cv  0.088301  0.075879  0.043628  0.057862  0.080463
471    cv       0.149803  0.150098  0.059911  0.078631  0.067806
       ause-cv  0.144645  0.145961  0.060239  0.076739  0.071390
global cv       0.140786  0.138462  0.057254  0.075555  0.068805
       ause-cv  0.125688  0.123544  0.053705  0.068155  0.068181

### SRP [pOK]

In [19]:
res, rcu = test(["srp"], lo_variance, hi_variance, projects, cfg)

creating
adding
adding 439
[rcu] adding StratifiedSRP for 439
[rcu] getting StratifiedSRP for ['439']
adding 471
[rcu] adding StratifiedSRP for 471
[rcu] getting StratifiedSRP for ['471']
[rcu] getting StratifiedSRP globally


In [20]:
res

P95     MeanH      Dens      Gini     Cover
439    srp       7.675560  4.135724  0.009542  0.001838  0.012452
       ause-srp  7.252322  3.869459  0.008440  0.001704  0.010853
471    srp       5.093515  2.487224  0.006748  0.001285  0.011395
       ause-srp  4.840088  2.347106  0.006197  0.001193  0.010400
global srp       5.443460  2.710645  0.007127  0.001360  0.011538
       ause-srp  5.354846  2.689091  0.006717  0.001310  0.010511

## Test RCU functionalities

In [21]:
del res, rcu

### Results stability

In [22]:
def test_stability(lo_variance, hi_variance, projects, cfg, N=2):
    print("creating")
    rcu = rcu = StratifiedRCU(
        num_variables=len(cfg["data_bands"]),
        # num_groups=len(projects),
        num_groups=N, # Debug
        num_bins=cfg["num_bins"],
        lo_variance=lo_variance,
        hi_variance=hi_variance
    )
    print("adding")
    results = {"orig": {}, "second": {}}
    for i, project in enumerate(projects):
        print("adding", project[0])
        rcu.add_project(*project)
        results["orig"][project[0]] = rcu.get_subset([project[0]])
    results["orig"]["global"] = rcu.get()
    for pid, _, _, _ in projects:
        results["second"][pid] = rcu.get_subset([pid])
    results["second"]["global"] = rcu.get()
    df1 = res2df(results["orig"], cfg)
    df2 = res2df(results["second"], cfg)
    df = pd.merge(df1, df2, left_index=True, right_index=True, suffixes=("_orig", "_second"))
    return results, df, rcu

In [24]:
res, df, rcu = test_stability(lo_variance, hi_variance, projects, cfg)

creating
adding
adding 439
[rcu] adding StratifiedMSE for 439
[rcu] adding StratifiedRMSE for 439
[rcu] adding StratifiedMAE for 439
[rcu] adding StratifiedMBE for 439
[rcu] adding StratifiedNLL for 439
[rcu] adding StratifiedUCE for 439
[rcu] adding StratifiedENCE for 439
[rcu] adding StratifiedCIAccuracy for 439
[rcu] adding StratifiedAUCE for 439
[rcu] adding StratifiedCv for 439
[rcu] adding StratifiedSRP for 439
[rcu] getting StratifiedMSE for ['439']
[rcu] getting StratifiedRMSE for ['439']
[rcu] getting StratifiedMAE for ['439']
[rcu] getting StratifiedMBE for ['439']
[rcu] getting StratifiedNLL for ['439']
[rcu] getting StratifiedUCE for ['439']
[rcu] getting StratifiedENCE for ['439']
[rcu] getting StratifiedCIAccuracy for ['439']
[rcu] getting StratifiedAUCE for ['439']
[rcu] getting StratifiedCv for ['439']
[rcu] getting StratifiedSRP for ['439']
adding 471
[rcu] adding StratifiedMSE for 471
[rcu] adding StratifiedRMSE for 471
[rcu] adding StratifiedMAE for 471
[rcu] adding 

In [25]:
df.iloc[:44]

P95_orig  MeanH_orig  Dens_orig  Gini_orig  Cover_orig  \
439 mse             0.325619    0.345869   0.014802   0.002589    0.019557   
    ause-mse        0.307903    0.324511   0.013128   0.002400    0.017197   
    rmse            0.549281    0.568683   0.116070   0.048601    0.128315   
    ause-rmse       0.523933    0.540405   0.107569   0.045919    0.118036   
    mae             0.435301    0.450296   0.092023   0.037767    0.101679   
    ause-mae        0.414987    0.427609   0.085085   0.035688    0.093075   
    mbe            -0.042112   -0.027191  -0.004988  -0.003574   -0.015277   
    ause-mbe       -0.040283   -0.025789  -0.003595  -0.003423   -0.013370   
    nll             0.962645    0.672108  -1.636143  -2.537406   -1.618837   
    ause-nll        0.918872    0.635548  -1.602038  -2.450142   -1.597655   
    uce             7.349940    3.789855   0.005263   0.000765    0.007109   
    ause-uce        6.944419    3.544948   0.004689   0.000701    0.006343   
    ence            0.000129    0.000124   0.000045   0.000062    0.000036   
    ause-ence       0.000084    0.000076   0.000022   0.000027    0.000018   
    ci90_accs       0.999993    0.999857   0.821685   0.852386    0.853149   
    ause-ci90_accs  0.959993    0.959859   0.788972   0.818340    0.821228   
    auce            0.368993    0.317995   0.067401   0.036826    0.065870   
    ause-auce       0.354288    0.305226   0.064118   0.035330    0.061623   
    cv              0.091181    0.077670   0.042320   0.059102    0.074839   
    ause-cv         0.088301    0.075879   0.043628   0.057862    0.080463   
    srp             7.675560    4.135724   0.009542   0.001838    0.012452   
    ause-srp        7.252322    3.869459   0.008440   0.001704    0.010853   
471 mse             0.267499    0.256655   0.014136   0.002227    0.022606   
    ause-mse        0.254306    0.242438   0.013027   0.002069    0.020753   
    rmse            0.497782    0.481926   0.113616   0.044666    0.143245   
    ause-rmse       0.476129    0.459646   0.106805   0.042359    0.133938   
    mae             0.387137    0.376022   0.089598   0.034760    0.113103   
    ause-mae        0.370243    0.358502   0.084124   0.032967    0.105471   
    mbe             0.033394    0.029016   0.000894   0.000052    0.006670   
    ause-mbe        0.031830    0.027660   0.000943   0.000015    0.005962   
    nll             0.742455    0.361909  -1.559775  -2.579001   -1.369919   
    ause-nll        0.709931    0.341542  -1.520492  -2.485340   -1.348215   
    uce             4.826016    2.230569   0.007389   0.000942    0.011216   
    ause-uce        4.585782    2.104669   0.006830   0.000876    0.010356   
    ence            0.000017    0.000014   0.000007   0.000007    0.000004   
    ause-ence       0.000012    0.000010   0.000004   0.000003    0.000003   
    ci90_accs       0.999852    0.999617   0.763954   0.808580    0.788883   
    ause-ci90_accs  0.959858    0.959630   0.734815   0.776314    0.760625   
    auce            0.355436    0.298897   0.104811   0.070676    0.097014   
    ause-auce       0.341217    0.286885   0.099554   0.067790    0.091232   
    cv              0.149803    0.150098   0.059911   0.078631    0.067806   
    ause-cv         0.144645    0.145961   0.060239   0.076739    0.071390   
    srp             5.093515    2.487224   0.006748   0.001285    0.011395   
    ause-srp        4.840088    2.347106   0.006197   0.001193    0.010400   

                    P95_second  MeanH_second  Dens_second  Gini_second  \
439 mse               0.325619      0.345869     0.014802     0.002589   
    ause-mse          0.307903      0.324511     0.013128     0.002400   
    rmse              0.549281      0.568683     0.116070     0.048601   
    ause-rmse         0.523933      0.540405     0.107569     0.045919   
    mae               0.435301      0.450296     0.092023     0.037767   
    ause-mae          0.414987      0.427609     0.085085     0.0

In [26]:
df.iloc[44:]

P95_orig  MeanH_orig  Dens_orig  Gini_orig  Cover_orig  \
global mse             0.275376    0.268746   0.014226   0.002276    0.022193   
       ause-mse        0.272725    0.267529   0.013505   0.002218    0.020252   
       rmse            0.504762    0.493684   0.113949   0.045200    0.141221   
       ause-rmse       0.491503    0.482542   0.108774   0.043827    0.131430   
       mae             0.393665    0.386088   0.089927   0.035168    0.111555   
       ause-mae        0.383314    0.377721   0.085812   0.034086    0.103479   
       mbe             0.023160    0.021398   0.000097  -0.000439    0.003696   
       ause-mbe        0.024351    0.022121   0.000220  -0.000377    0.003483   
       nll             0.772297    0.403950  -1.570125  -2.573364   -1.403655   
       ause-nll        0.752109    0.405491  -1.511714  -2.458143   -1.388013   
       uce             5.168084    2.441899   0.007100   0.000916    0.010655   
       ause-uce        5.082120    2.421562   0.006788   0.000909    0.009742   
       ence            0.000017    0.000016   0.000007   0.000006    0.000005   
       ause-ence       0.000010    0.000009   0.000003   0.000002    0.000002   
       ci90_accs       0.999871    0.999649   0.771778   0.814517    0.797593   
       ause-ci90_accs  0.959874    0.959667   0.740188   0.780727    0.769884   
       auce            0.357274    0.301485   0.099741   0.066089    0.092793   
       ause-auce       0.342780    0.289376   0.096202   0.064276    0.086486   
       cv              0.140786    0.138462   0.057254   0.075555    0.068805   
       ause-cv         0.125688    0.123544   0.053705   0.068155    0.068181   
       srp             5.443460    2.710645   0.007127   0.001360    0.011538   
       ause-srp        5.354846    2.689091   0.006717   0.001310    0.010511   

                       P95_second  MeanH_second  Dens_second  Gini_second  \
global mse               0.275376      0.268746     0.014226     0.002276   
       ause-mse          0.272725      0.267529     0.013505     0.002218   
       rmse              0.504762      0.493684     0.113949     0.045200   
       ause-rmse         0.491503      0.482542     0.108774     0.043827   
       mae               0.393665      0.386088     0.089927     0.035168   
       ause-mae          0.383314      0.377721     0.085812     0.034086   
       mbe               0.023160      0.021398     0.000097    -0.000439   
       ause-mbe          0.024351      0.022121     0.000220    -0.000377   
       nll               0.772297      0.403950    -1.570125    -2.573364   
       ause-nll          0.752109      0.405491    -1.511714    -2.458143   
       uce               5.168084      2.441899     0.007100     0.000916   
       ause-uce          5.082120      2.421562     0.006788     0.000909   
       ence              0.000017      0.000016     0.000007     0.000006   
       ause-ence         0.000010      0.000009     0.000003     0.000002   
       ci90_accs         0.999871      0.999649     0.771778     0.814517   
       ause-ci90_accs    0.959874      0.959667     0.740188     0.780727   
       auce              0.357274      0.301485     0.099741     0.066089   
       ause-auce         0.342780      0.289376     0.096202     0.064276   
       cv                0.140786      0.138462     0.057254     0.075555   
       ause-cv           0.125688      0.123544     0.053705     0.068155   
       srp               5.443460      2.710645     0.007127     0.001360   
       ause-srp          5.354846      2.689091     0.006717     0.001310   

                       Cover_second  
global mse                 0.022193  
       ause-mse            0.020252  
       rmse                0.141221  
       ause-rmse           0.131430  
       mae                 0.111555  
       ause-mae            0.103479  
       mbe                 0.003696  
       ause-mbe            0.003483  
       nll                -1.403655  
       

### Copy

In [27]:
def test_copy(lo_variance, hi_variance, projects, cfg, N=2):
    print("creating")
    rcu = rcu = StratifiedRCU(
        num_variables=len(cfg["data_bands"]),
        # num_groups=len(projects),
        num_groups=N, # Debug
        num_bins=cfg["num_bins"],
        lo_variance=lo_variance,
        hi_variance=hi_variance
    )
    print("adding")
    results = {"orig": {}, "copy": {}}
    for i, project in enumerate(projects):
        print("adding", project[0])
        rcu.add_project(*project)
        results["orig"][project[0]] = rcu.get_subset([project[0]])
    results["orig"]["global"] = rcu.get()
    rcu_c = rcu.copy()
    for pid, _, _, _ in projects:
        results["copy"][pid] = rcu_c.get_subset([pid])
    results["copy"]["global"] = rcu_c.get()
    df1 = res2df(results["orig"], cfg)
    df2 = res2df(results["copy"], cfg)
    df = pd.merge(df1, df2, left_index=True, right_index=True, suffixes=("_orig", "_copy"))
    return results, df, rcu, rcu_c

In [28]:
res, df, rcu1, rcu2 = test_copy(lo_variance, hi_variance, projects, cfg)

creating
adding
adding 439
[rcu] adding StratifiedMSE for 439
[rcu] adding StratifiedRMSE for 439
[rcu] adding StratifiedMAE for 439
[rcu] adding StratifiedMBE for 439
[rcu] adding StratifiedNLL for 439
[rcu] adding StratifiedUCE for 439
[rcu] adding StratifiedENCE for 439
[rcu] adding StratifiedCIAccuracy for 439
[rcu] adding StratifiedAUCE for 439
[rcu] adding StratifiedCv for 439
[rcu] adding StratifiedSRP for 439
[rcu] getting StratifiedMSE for ['439']
[rcu] getting StratifiedRMSE for ['439']
[rcu] getting StratifiedMAE for ['439']
[rcu] getting StratifiedMBE for ['439']
[rcu] getting StratifiedNLL for ['439']
[rcu] getting StratifiedUCE for ['439']
[rcu] getting StratifiedENCE for ['439']
[rcu] getting StratifiedCIAccuracy for ['439']
[rcu] getting StratifiedAUCE for ['439']
[rcu] getting StratifiedCv for ['439']
[rcu] getting StratifiedSRP for ['439']
adding 471


/scratch/ewalt/pdm/rs-uncertainty/src/metrics.py:250: RuntimeWarning: invalid value encountered in true_divide
  result = np.abs(result)/np.sqrt(np.nansum(histogram*arrs[0], axis=self.groups_axis, keepdims=True))


[rcu] adding StratifiedMSE for 471
[rcu] adding StratifiedRMSE for 471
[rcu] adding StratifiedMAE for 471
[rcu] adding StratifiedMBE for 471
[rcu] adding StratifiedNLL for 471
[rcu] adding StratifiedUCE for 471
[rcu] adding StratifiedENCE for 471
[rcu] adding StratifiedCIAccuracy for 471
[rcu] adding StratifiedAUCE for 471
[rcu] adding StratifiedCv for 471
[rcu] adding StratifiedSRP for 471
[rcu] getting StratifiedMSE for ['471']
[rcu] getting StratifiedRMSE for ['471']
[rcu] getting StratifiedMAE for ['471']
[rcu] getting StratifiedMBE for ['471']
[rcu] getting StratifiedNLL for ['471']
[rcu] getting StratifiedUCE for ['471']
[rcu] getting StratifiedENCE for ['471']
[rcu] getting StratifiedCIAccuracy for ['471']
[rcu] getting StratifiedAUCE for ['471']
[rcu] getting StratifiedCv for ['471']
[rcu] getting StratifiedSRP for ['471']
[rcu] getting StratifiedMSE globally
[rcu] getting StratifiedRMSE globally
[rcu] getting StratifiedMAE globally
[rcu] getting StratifiedMBE globally
[rcu] ge

In [29]:
df.iloc[:44]

P95_orig  MeanH_orig  Dens_orig  Gini_orig  Cover_orig  \
439 mse             0.325619    0.345869   0.014802   0.002589    0.019557   
    ause-mse        0.307903    0.324511   0.013128   0.002400    0.017197   
    rmse            0.549281    0.568683   0.116070   0.048601    0.128315   
    ause-rmse       0.523933    0.540405   0.107569   0.045919    0.118036   
    mae             0.435301    0.450296   0.092023   0.037767    0.101679   
    ause-mae        0.414987    0.427609   0.085085   0.035688    0.093075   
    mbe            -0.042112   -0.027191  -0.004988  -0.003574   -0.015277   
    ause-mbe       -0.040283   -0.025789  -0.003595  -0.003423   -0.013370   
    nll             0.962645    0.672108  -1.636143  -2.537406   -1.618837   
    ause-nll        0.918872    0.635548  -1.602038  -2.450142   -1.597655   
    uce             7.349940    3.789855   0.005263   0.000765    0.007109   
    ause-uce        6.944419    3.544948   0.004689   0.000701    0.006343   
    ence            0.000129    0.000124   0.000045   0.000062    0.000036   
    ause-ence       0.000084    0.000076   0.000022   0.000027    0.000018   
    ci90_accs       0.999993    0.999857   0.821685   0.852386    0.853149   
    ause-ci90_accs  0.959993    0.959859   0.788972   0.818340    0.821228   
    auce            0.368993    0.317995   0.067401   0.036826    0.065870   
    ause-auce       0.354288    0.305226   0.064118   0.035330    0.061623   
    cv              0.091181    0.077670   0.042320   0.059102    0.074839   
    ause-cv         0.088301    0.075879   0.043628   0.057862    0.080463   
    srp             7.675560    4.135724   0.009542   0.001838    0.012452   
    ause-srp        7.252322    3.869459   0.008440   0.001704    0.010853   
471 mse             0.267499    0.256655   0.014136   0.002227    0.022606   
    ause-mse        0.254306    0.242438   0.013027   0.002069    0.020753   
    rmse            0.497782    0.481926   0.113616   0.044666    0.143245   
    ause-rmse       0.476129    0.459646   0.106805   0.042359    0.133938   
    mae             0.387137    0.376022   0.089598   0.034760    0.113103   
    ause-mae        0.370243    0.358502   0.084124   0.032967    0.105471   
    mbe             0.033394    0.029016   0.000894   0.000052    0.006670   
    ause-mbe        0.031830    0.027660   0.000943   0.000015    0.005962   
    nll             0.742455    0.361909  -1.559775  -2.579001   -1.369919   
    ause-nll        0.709931    0.341542  -1.520492  -2.485340   -1.348215   
    uce             4.826016    2.230569   0.007389   0.000942    0.011216   
    ause-uce        4.585782    2.104669   0.006830   0.000876    0.010356   
    ence            0.000017    0.000014   0.000007   0.000007    0.000004   
    ause-ence       0.000012    0.000010   0.000004   0.000003    0.000003   
    ci90_accs       0.999852    0.999617   0.763954   0.808580    0.788883   
    ause-ci90_accs  0.959858    0.959630   0.734815   0.776314    0.760625   
    auce            0.355436    0.298897   0.104811   0.070676    0.097014   
    ause-auce       0.341217    0.286885   0.099554   0.067790    0.091232   
    cv              0.149803    0.150098   0.059911   0.078631    0.067806   
    ause-cv         0.144645    0.145961   0.060239   0.076739    0.071390   
    srp             5.093515    2.487224   0.006748   0.001285    0.011395   
    ause-srp        4.840088    2.347106   0.006197   0.001193    0.010400   

                    P95_copy  MeanH_copy  Dens_copy  Gini_copy  Cover_copy  
439 mse             0.325619    0.345869   0.014802   0.002589    0.019557  
    ause-mse        0.307903    0.324511   0.013128   0.002400    0.017197  
    rmse            0.549281    0.568683   0.116070   0.048601    0.128315  
    ause-rmse       0.523933    0.540405   0.107569   0.045919    0.118036  
    mae             0.435301    0.450296   0.092023   0.037767    0.101679  
    ause-mae        0.414987    0.427609   0.08

In [30]:
df.iloc[44:]

P95_orig  MeanH_orig  Dens_orig  Gini_orig  Cover_orig  \
global mse             0.275376    0.268746   0.014226   0.002276    0.022193   
       ause-mse        0.272725    0.267529   0.013505   0.002218    0.020252   
       rmse            0.504762    0.493684   0.113949   0.045200    0.141221   
       ause-rmse       0.491503    0.482542   0.108774   0.043827    0.131430   
       mae             0.393665    0.386088   0.089927   0.035168    0.111555   
       ause-mae        0.383314    0.377721   0.085812   0.034086    0.103479   
       mbe             0.023160    0.021398   0.000097  -0.000439    0.003696   
       ause-mbe        0.024351    0.022121   0.000220  -0.000377    0.003483   
       nll             0.772297    0.403950  -1.570125  -2.573364   -1.403655   
       ause-nll        0.752109    0.405491  -1.511714  -2.458143   -1.388013   
       uce             5.168084    2.441899   0.007100   0.000916    0.010655   
       ause-uce        5.082120    2.421562   0.006788   0.000909    0.009742   
       ence            0.000017    0.000016   0.000007   0.000006    0.000005   
       ause-ence       0.000010    0.000009   0.000003   0.000002    0.000002   
       ci90_accs       0.999871    0.999649   0.771778   0.814517    0.797593   
       ause-ci90_accs  0.959874    0.959667   0.740188   0.780727    0.769884   
       auce            0.357274    0.301485   0.099741   0.066089    0.092793   
       ause-auce       0.342780    0.289376   0.096202   0.064276    0.086486   
       cv              0.140786    0.138462   0.057254   0.075555    0.068805   
       ause-cv         0.125688    0.123544   0.053705   0.068155    0.068181   
       srp             5.443460    2.710645   0.007127   0.001360    0.011538   
       ause-srp        5.354846    2.689091   0.006717   0.001310    0.010511   

                       P95_copy  MeanH_copy  Dens_copy  Gini_copy  Cover_copy  
global mse             0.275376    0.268746   0.014226   0.002276    0.022193  
       ause-mse        0.272725    0.267529   0.013505   0.002218    0.020252  
       rmse            0.504762    0.493684   0.113949   0.045200    0.141221  
       ause-rmse       0.491503    0.482542   0.108774   0.043827    0.131430  
       mae             0.393665    0.386088   0.089927   0.035168    0.111555  
       ause-mae        0.383314    0.377721   0.085812   0.034086    0.103479  
       mbe             0.023160    0.021398   0.000097  -0.000439    0.003696  
       ause-mbe        0.024351    0.022121   0.000220  -0.000377    0.003483  
       nll             0.772297    0.403950  -1.570125  -2.573364   -1.403655  
       ause-nll        0.752109    0.405491  -1.511714  -2.458143   -1.388013  
       uce             5.168084    2.441899   0.007100   0.000916    0.010655  
       ause-uce        5.082120    2.421562   0.006788   0.000909    0.009742  
       ence            0.000017    0.000016   0.000007   0.000006    0.000005  
       ause-ence       0.000010    0.000009   0.000003   0.000002    0.000002  
       ci90_accs       0.999871    0.999649   0.771778   0.814517    0.797593  
       ause-ci90_accs  0.959874    0.959667   0.740188   0.780727    0.769884  
       auce            0.357274    0.301485   0.099741   0.066089    0.092793  
       ause-auce       0.342780    0.289376   0.096202   0.064276    0.086486  
       cv              0.140786    0.138462   0.057254   0.075555    0.068805  
       ause-cv         0.125688    0.123544   0.053705   0.068155    0.068181  
       srp             5.443460    2.710645   0.007127   0.001360    0.011538  
       ause-srp        5.354846    2.689091   0.006717   0.001310    0.010511

### Upsample

In [ ]:
def test_upsample(rcu, ks):
    # visually: for a single project
    # plot all X or (X1, X2)
    pass